In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.


*Load specific modules used here:*

In [2]:
os.chdir(d['py'])
import mProduction, modelData
from gmsPython import nestingTree
from valueShares import nestedShares

*Load test data:*

In [3]:
t0 = 2019
name = 'TestModelData' #
db = GpyDB(os.path.join(d['data'], name), name = 'test', ws = d['work'])
ws = db.ws

Subset data to only include a specific production sector and two years:

In [4]:
# AggDB.subsetDB(db, db('s_p'))
AggDB.subsetDB(db, db('s_p')[1:2])
AggDB.subsetDB(db, db('t')[0:2])
modelData.addTimeToDB(t0, t0+1, db) # add a number of relevant time subsets 
db0 = db.copy()

Remove durables:

In [5]:
db['qS'] = db('qS')-(adj.rc_pd(db('qD'), db('dur_p')) * db('pD_dur')).groupby(['t','s']).sum()
db['qD'] = adj.rc_pd(db('qD'), ('not', db('dur_p')))
db['nestProduction'] = adj.rc_pd(db('nestProduction'), ('not', db('dur_p').rename('nn')))

*Set up nesting structure from nesting tree:*

In [6]:
nest = nestingTree.Tree('P', tree = db('nestProduction').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'P', trees = {t.name: t for t in [nest]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

*Get starting values for $\mu$ using value shares:*

In [7]:
v = nestedShares(nestAgg, ws = ws)
db_p = v(db)

Use share parameters and values to get initial values for intermediates:

In [8]:
db.aom(db_p('mu').xs(t0), name = 'mu')
db.aom(adj.rc_pd(db_p('vD'), nestAgg.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Initialize model instance:

In [9]:
m = mProduction.StaticNCES(nestAgg, partial = True, initFromGms = 'initFirmValueBlock')

Add data and initialize other stuff with default options:

In [10]:
m.initStuff(db = db)

Solve:

In [11]:
dbsolve = m.solve(state = 'C')